# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [164]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import AdaBoostClassifier

[nltk_data] Downloading package punkt to /home/brunom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brunom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)

In [91]:
df.head()

,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
df.iloc[:,4:] = df.iloc[:,4:].astype('int32')

In [93]:
list_melt = df.iloc[:,4:].columns.tolist()
dict_classes = {key:val for val,key in enumerate(df.iloc[:,4:].columns.tolist())}
df_target = df.iloc[:,4:].apply(lambda x: np.argwhere(x==1), axis = 1).copy()
labels = df_target.apply(lambda x: x.flatten().tolist()).values.tolist()

In [98]:
df.iloc[:,4:].shape

(26216, 36)

In [185]:
df_target.shape

(26216,)

### 2. Write a tokenization function to process your text data

In [99]:
stemmer = PorterStemmer()
def tokenize(sentence):
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [stemmer.stem(word) for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [100]:
df_clean_message = df.message.apply(lambda x: " ".join(tokenize(x)))

In [101]:
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
#vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1,1))
#vectorizer = TfidfVectorizer(stop_words="english", sublinear_tf=True, norm='l1')
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
final_features = pd.DataFrame(vectorizer.fit_transform(df_clean_message).toarray(), 
                              columns=vectorizer.get_feature_names())
#display(vectorizer.get_feature_names())
display(final_features.head())

,aa,aai,ab,ababa,abad,abandon,abba,abdomin,abdomin pain,abduct,...,zhejiang provinc,zhouqu,zimbabw,zimbabwean,zinc,zinc roof,zone,zoonot,zoonot diseas,zy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
#vectorizer2 = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
vectorizer2 = CountVectorizer(min_df= 1,stop_words="english", ngram_range=(1, 4))
X = vectorizer2.fit_transform(df_clean_message)
y = MultiLabelBinarizer().fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
clf = OneVsRestClassifier(MultinomialNB())
#clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.1664355062413315

In [189]:
y.shape

(26216, 35)

In [128]:
#X = vector_features
y2 = df.iloc[:,4:-1].values
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.33, random_state=32)

In [163]:
def tokenize(sentence):
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [word for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 
df_clean_message = df.message.apply(lambda x: " ".join(tokenize(x)))
#vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 2))
vectorizer = TfidfVectorizer(min_df= 5, stop_words="english", sublinear_tf=True, norm='l1', ngram_range=(1, 2))
vector_features = vectorizer.fit_transform(df_clean_message).toarray()
X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

#clf = OneVsRestClassifier(SVC(probability=True))
#clf = OneVsRestClassifier(BernoulliNB())
clf = OneVsRestClassifier(AdaBoostClassifier())
clf.fit(X_train, y_train)
clf.fit(X_train, y_trainb

accuracy_score(y_test, predictions)

0.2311604253351826

In [173]:
df.iloc[:,4:].sum(axis=0)

aid_centers                 309
aid_related               10860
buildings                  1333
child_alone                   0
clothing                    405
cold                        530
death                      1194
direct_report              5075
earthquake                 2455
electricity                 532
fire                        282
floods                     2155
food                       2923
hospitals                   283
infrastructure_related     1705
medical_help               2084
medical_products           1313
military                    860
missing_people              298
money                       604
offer                       118
other_aid                  3446
other_infrastructure       1151
other_weather              1376
refugees                    875
related                   20282
request                    4474
search_and_rescue           724
security                    471
shelter                    2314
shops                       120
storm   

In [175]:
len(df.iloc[:,4:].columns)

36

In [182]:
accuracy_score(y_test[:,0], predictions[:,0])

0.9878640776699029

In [184]:
y_test.shape

(8652, 35)

In [190]:
len(df.iloc[:,4:].columns)
[accuracy_score(y_test[:,column],predictions[:,column]) for column in range(len(df.iloc[:,4:].columns)-1)]

[0.9878640776699029,
 0.7457235321312992,
 0.9617429496070273,
 0.9872861766065649,
 0.9843966712898752,
 0.9664817383263985,
 0.8542533518261674,
 0.9696024040684235,
 0.9812760055478502,
 0.9912159038372631,
 0.9555016181229773,
 0.9435968562182154,
 0.9864771151178918,
 0.9294960702727693,
 0.9254507628294036,
 0.9526121128062875,
 0.9699491447064262,
 0.9878640776699029,
 0.9769995376791494,
 0.9939898289412853,
 0.8646555709662506,
 0.9520342117429496,
 0.9484512251502543,
 0.9684466019417476,
 0.7786638927415627,
 0.8919325011558021,
 0.970873786407767,
 0.9823162274618585,
 0.9469486823855756,
 0.994914470642626,
 0.9348127600554785,
 0.9942209893666204,
 0.9586222838650024,
 0.9605871474803513,
 0.8788719371243643]

In [175]:
clf = OneVsRestClassifier(MultinomialNB())
clf.fit(X_train, y_train)
preds = clf.predict(X_train) 

ValueError: Multioutput target data is not supported with label binarization

In [ ]:
OneVsRestClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X) 
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(X_train, y_train)


ValueError: bad input shape (17564, 37)

## Code Bank

In [61]:
# beautiful solution
#df.message.apply(lambda x: " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).split() 
#          if word not in stop_words]) )